In [ ]:
# --------------------------------------------------
# PhoMemes 2023 Evaluation Script
#  - For Disinformation Challenge Evaluation
#
# Used to evaluate PhoMeme challenge submission for
#. influence campaigns
# --------------------------------------------------
version = 1.0 # Notebook Version Number
edition = "c1"

import os
cwd = os.getcwd()

In [ ]:
import json

import numpy as np
import pandas as pd

from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

In [ ]:
model_info = None
with open("metadata.json", "r") as in_file:
    model_info = json.load(in_file)
    
model_info

In [ ]:
groundtruth_df = pd.read_csv("c1.groundtruth.csv", index_col="encoded_id")
run_df = pd.read_csv("run.csv", index_col="user_id")

In [ ]:
joined_df = run_df.join(groundtruth_df, how="inner", lsuffix="_pred", rsuffix="_truth")

In [ ]:
joined_df.columns

In [ ]:
disinfo_agent_prediction = [True if a == 1 else False for a in joined_df["inauthentic_pred"]]
disinfo_agent_truth = [True if a == 1 else False for a in joined_df["inauthentic_truth"]]

ac_score_overall = accuracy_score(disinfo_agent_prediction, disinfo_agent_truth)
pr_score_overall = precision_score(disinfo_agent_prediction, disinfo_agent_truth)
rc_score_overall = recall_score(disinfo_agent_prediction, disinfo_agent_truth)

f1_score_overall = f1_score(disinfo_agent_prediction, disinfo_agent_truth, zero_division=1)

In [ ]:
print("Accuracy:", ac_score_overall)
print("Precision:", pr_score_overall)
print("Recall:", rc_score_overall)
print("F1:", f1_score_overall)

In [ ]:
acs = []
prs = []
rcs = []
f1s = []

for i,j in joined_df["campaign_truth"].value_counts().items():
    print(i,j)
    
    this_pred = joined_df['campaign_pred'] == i
    this_true = joined_df['campaign_truth'] == i
    
    ac_score_this = accuracy_score(this_pred, this_true)
    acs.append(ac_score_this)
    
    pr_score_this = precision_score(this_pred, this_true, zero_division=0)
    prs.append(pr_score_this)
    
    rc_score_this = recall_score(this_pred, this_true, zero_division=0)
    rcs.append(rc_score_this)
    
    f1_score_this = f1_score(this_pred, this_true, zero_division=0)
    f1s.append(f1_score_this)
    
    print("[%s] Ac:" % i, ac_score_this)
    print("[%s] Pr:" % i, pr_score_this)
    print("[%s] Rc:" % i, rc_score_this)
    print("[%s] F1:" % i, f1_score_this)
    
    print("*" * 20)
    

In [ ]:
ac_mean = np.mean(acs)
pr_mean = np.mean(prs)
rc_mean = np.mean(rcs)
f1_mean = np.mean(f1s)

print("Macro-Average Ac:", ac_mean)
print("Macro-Average Pr:", pr_mean)
print("Macro-Average Rc:", rc_mean)
print("Macro-Average F1:", f1_mean)

In [ ]:
sub_date_ = cwd.partition("submissions/")[-1].partition("/")[-1]
sub_date_ = sub_date_.partition("-")[0]
sub_date = "%s/%s/%s" % (sub_date_[:4], sub_date_[4:6], sub_date_[6:])


In [ ]:
rows = [{
    "runtag": "%s-%s-%s" % (model_info["organization"], model_info["runtag"], model_info["date"]),
    "date": sub_date,
    "organization": model_info["organization"],
    "model_description": model_info["model_description"],
    "type": model_info["type"],
    "paper": model_info["paper"],
    "code": model_info["code"],
    "accuracy_overall": ac_score_overall,
    "precision_overall": pr_score_overall,
    "recall_overall": rc_score_overall,
    "f1_overall": f1_score_overall,
    "accuracy_macro": ac_mean,
    "precision_macro": pr_mean,
    "recall_macro": rc_mean,
    "f1_macro": f1_mean,
}]

leaderboard_df = pd.DataFrame(rows).set_index("runtag")
leaderboard_df

In [ ]:
leaderboard_df.to_csv("leaderboard.csv")